In [1]:
#scrape urban turf
%pip install pandas
%pip install requests
import requests
%pip install beautifulsoup4
%pip install geopy
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time
import certifi

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
page = requests.get(
    "https://dc.urbanturf.com/pipeline?search_string=&project_type=0&status=0&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes&limit=1000")
soup = BeautifulSoup(page.content, 'html.parser')
page_body = soup.body

labels = [item for item in page_body.findAll("p")]
list_lab = [i.text.strip() for i in labels][1:] 
list_lab = [word.replace('\n','') for word in list_lab]

torep = {"Location:            ":"", "Project type:            ":"","Status:            ":"", "Size:            ":""} #titles of list
torep = dict((re.escape(k), v) for k, v in torep.items()) #turn into dictionary using 'torep' value as keys
pattern = re.compile("|".join(torep.keys())) #joins with "|"

desc = []

for i in range(0, len(list_lab)):
    a = list_lab[i] #first value in list
    text = pattern.sub(lambda m:torep[re.escape(m.group(0))], a) #substitute the pattern in each value of the list with ""
    desc.append(text)

seq = range(0, len(desc))

urban_turf= pd.DataFrame(
    {"Address": [desc[i] for i in seq[0::5]],
     "Neighborhood": [desc[i] for i in seq[1::5]],
     "Project.Type":[desc[i] for i in seq[2::5]],
     "Status":[desc[i] for i in seq[3::5]],
     "Size":[desc[i] for i in seq[4::5]],
     "Pipeline Link": None,
     'Neighborhood:':None,
     'Pipeline':None,
     'Project type:': None, 
     'No. of units:': None, 
     'Types of units:': None, 
     'Unit sizes:':None, 
     'Amenities:':None, 
     'Pricing:':None, 
     'Website:':None, 
     'Architect:':None, 
     'Last updated:':None, 
     'Address:':None, 
     'Status:':None, 
     'First move-ins:':None, 
     "Latitude":None, 
     "Longitude":None})    

#can I get the pipeline link from scraping above?

def search_pipe_link(row):
    ut_address_col = str(row['Address'])
    projecttype_col = str(row['Project.Type'])
    status_col = str(row['Status'])
    toreplace = "&more_or_less=0&number_of_units=&number_of_units_selector=0&city=0&state=0&zip=0&order_by=last_updated&direction=desc&filtered=Yes"
    time.sleep(1.25)
    searchlink = "https://dc.urbanturf.com/pipeline?search_string=" + ut_address_col.replace(" ", "_" ) + "&project_type=" + projecttype_col.replace(" ", "+").replace("&", "%26") + "&status=" + status_col.replace(",", "%2C").replace(" ", "+") + toreplace
    page = requests.get(searchlink)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_body = soup.body
    table = page_body.find_all('div', class_ = 'pipeline-item clickable')
    if table:
        tag = table[0].find('a').get('href')
        return tag
    else:
        return None

urban_turf['Pipeline Link'] = urban_turf.apply(search_pipe_link, axis=1)
urban_turf = urban_turf.dropna(subset=['Pipeline Link']) #would like to encorporate into function
urban_turf = urban_turf.reset_index(drop=True)
column_names = urban_turf.columns

urban_turf.loc[urban_turf['Address'] == "The MO", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/715/The_MO"
urban_turf.loc[urban_turf['Address'] == "Valo", 'Pipeline Link'] = "https://dc.urbanturf.com/pipeline/551/Valo"

def scrape_pipelinelink(row):
    page = requests.get(row["Pipeline Link"])
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('div', class_ = 'block') 
    txt = str(table)
    txt = txt.replace('\r', '').replace('\n', '')
    txt = txt.split("<p>")
    txt = [x for x in txt if '<span class="label">' in x]
    txt = [txt.split("</span>") for txt in txt]

    key = [txt[i][0] for i in list(range(0,len(txt)))]
    txt = [txt[i][1] for i in list(range(0,len(txt)))]
    key = [x.replace('<span class="label">', "") for x in key]
    df_dictionary = pd.DataFrame([ dict(zip(key, txt))])

    for key, txt in df_dictionary.items():
        if key in column_names:
            row[key] = df_dictionary[key][0]

urban_turf.apply(scrape_pipelinelink, axis=1)

urban_turf['No. of units:'] = [x.replace(" ", "").replace("</p>", "") for x in urban_turf['No. of units:']]
urban_turf.loc[urban_turf['Architect:'].notnull(), 'Architect:'] = [x.replace("\t", "").replace("</p>", "") for x in urban_turf.loc[urban_turf['Architect:'].notnull(), 'Architect:']]
urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:'] = [re.search(r'".*\"\sstyle', x).group(0).replace('" style', "").replace('"', "") for x in urban_turf.loc[urban_turf['Website:'].notnull(), 'Website:']]
urban_turf.loc[urban_turf['Status:'].notnull(), 'Status:'] =  [re.sub(r'</p>.*', "", x) for x in urban_turf.loc[urban_turf['Status:'].notnull(), 'Status:']]
urban_turf.loc[urban_turf['Address:'].notnull(), 'Address:'] =  [re.sub(r'</p>.*', "", x) for x in urban_turf.loc[urban_turf['Address:'].notnull(), 'Address:']]

substitutions = [
    (r'<br/>', ''),
    (r'\xa0', ' '),
    (r'&amp;', '&'),
    #(r' +', "")
]

def replace_text(row):
    for old_pattern, new_string in substitutions:
        row['Address:'] = re.sub(old_pattern, new_string, row['Address:'])

urban_turf.apply(replace_text, axis=1)

urban_turf['Address:'] = [re.sub(r'( {2,})(\.*)( {2,})(\.*)', r'\2,\3\4', x) for x in urban_turf['Address:']] #why are there commas on each side?

def get_lat_long(address):
    geolocator = Nominatim(user_agent="my_geocoder_app")
    time.sleep(1.1)
    location = geolocator.geocode(address)

    if location:
        return location.latitude, location.longitude
    else:
        return None
    
for i in range(0, len(urban_turf)):
    try:
        c = urban_turf['Address:'][i]
        d = re.sub("-[\d]*", "", c)
        a = get_lat_long(d)
        urban_turf['Latitude'][i] =  a[0]
        urban_turf['Longitude'][i] =  a[1]

    except Exception:
        pass
    
urban_turf['No. of units:'] = pd.to_numeric(urban_turf['No. of units:'], errors='coerce')


'\ndef get_lat_long(address):\n    geolocator = Nominatim(user_agent="my_geocoder_app")\n    time.sleep(1.1)\n    location = geolocator.geocode(address)\n\n    if location:\n        return location.latitude, location.longitude\n    else:\n        return None\n    \nurban_turf[\'Latitude\'] = ""\nurban_turf[\'Longitude\'] = ""\n\nfor i in range(0, len(urban_turf)):\n    try:\n        c = urban_turf[\'Address:\'][i]\n        d = re.sub("-[\\d]*", "", c)\n        a = get_lat_long(d)\n        urban_turf[\'Latitude\'][i] =  a[0]\n        urban_turf[\'Longaitude\'][i] =  a[1]\n\n    except Exception:\n        pass\nurban_turf[\'No. of units:\'] = pd.to_numeric(urban_turf[\'No. of units:\'], errors=\'coerce\')\n\n'